# Support Vector Machines


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from helpers import get_scores
from sklearn.grid_search import RandomizedSearchCV
from transformers import transformer
from data_builder import load_test_data, load_dev_data, load_small_dev_data

df, target = load_dev_data()
X = transformer.fit_transform(df)

print(X.shape)

(80993, 273)



Conjunto a probar

1)C in range [0:20] Dice el manual:Por default es uno. If you have a lot of noisy observations you should decrease it. It corresponds to regularize more the estimation.

2)Kernels {'rbf ' ,' linear' ,' poly' ,'sigmoid'}  

3) para poly podemos hacer un range de degrees, esa opcion va a ser ignorada por los otros kernels asi que podemos hacer degree in range[2:5]. Podemos elegis=r ademas si agregamos un coef0, o sea el termino independiente, no veo por que seria importnte por ahora eso.

4)(REVISAR)Por default hay un parametro epsilon=0.1. Epsilon indica un rango de distancia que no sera considerado como penalidad, si el vector esta a espilon--ver

decision_function(X)[source]
Distance of the samples X to the separating hyperplane.
Parameters:	
X : array-like, shape (n_samples, n_features)
returns:	
X : array-like, shape (n_samples, n_classes * (n_classes-1) / 2)
Returns the decision function of the sample for each class in the model. If decision_function_shape=’ovr’, the shape is (n_samples, n_classes)

-Cuando fiteamos este modelo podemos elegir un sample weight, pero no creo que lo necesitemos ya que estan balanceados.



In [2]:
from sklearn.svm import SVC, LinearSVC

help(LinearSVC)

Help on class LinearSVC in module sklearn.svm.classes:

class LinearSVC(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.feature_selection.from_model._LearntSelectorMixin, sklearn.linear_model.base.SparseCoefMixin)
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  C : float, optional (default=1.0)
 |      Penalty parameter C of the error term.
 |  
 |  loss : string, 'hinge' or 'squared_hinge' (default='squared_hinge')
 |      Specifies the loss function. 'hinge' is the st

In [5]:
from search import find_best_classifier
from sklearn.grid_search import GridSearchCV

options = {
    'C': np.arange(0.2, 5.0, 0.4),
}

search_options = {
    'cv': 3,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 5,
}

search = RandomizedSearchCV(LinearSVC(), options, verbose=3, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)  


Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] C=2.2 ...........................................................
[CV] C=2.2 ...........................................................
[CV] C=2.2 ...........................................................
[CV] C=4.2 ...........................................................
[CV] .................................. C=2.2, score=0.953482 -  27.3s
[CV] C=4.2 ...........................................................
[CV] .................................. C=2.2, score=0.897057 -  35.0s
[CV] C=4.2 ...........................................................
[CV] .................................. C=4.2, score=0.953482 -  47.0s
[CV] C=4.6 ...........................................................
[CV] .................................. C=2.2, score=0.809139 -  50.1s
[CV] C=4.6 ...........................................................
[CV] .................................. C=4.2, score=0.900411 -  36.3s
[CV] C=4.6 ......

[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.2min finished


Mejores parámetros: {'C': 4.6000000000000014}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
LinearSVC,0.900729,0.878556,0.8751,0.850889,0.878556


# Con Kernel RBF



In [ ]:
from search import find_best_classifier
from sklearn.grid_search import GridSearchCV

options = {
    'kernel': ['rbf'],
    'C': np.arange(0.2, 5.0, 0.4),
    'gamma': np.arange(0.01, 1.00, 0.01),
}

search_options = {
    'cv': 3,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 10,
}

search = RandomizedSearchCV(SVC(), options, verbose=3, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)  


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] kernel=rbf, C=1.0, gamma=0.75 ...................................
[CV] kernel=rbf, C=1.0, gamma=0.75 ...................................
[CV] kernel=rbf, C=1.0, gamma=0.75 ...................................
[CV] kernel=rbf, C=4.2, gamma=0.15 ...................................
